### Setting up Kaggle API

In [ ]:
import shutil
import os

src = "/content/drive/MyDrive/Colab Notebooks/kaggle.json"
dst_folder = "/root/.kaggle"
dst = os.path.join(dst_folder, "kaggle.json")

os.makedirs(dst_folder, exist_ok=True)

shutil.move(src, dst)

os.chmod(dst, 600)

print("✅ kaggle.json moved successfully!")


In [2]:
!pip install kaggle
!kaggle datasets list


### Downloading Flickr8k Dataset

In [3]:
!kaggle datasets download -d adityajn105/flickr8k


In [ ]:
import zipfile

with zipfile.ZipFile("/content/flickr8k.zip", "r") as zip_ref:
    zip_ref.extractall("flickr8k")

os.remove("flickr8k.zip")

!ls flickr8k


### 1️⃣ Install Dependencies & Load Dataset

In [5]:
!pip install transformers datasets accelerate evaluate --quiet


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

CAPTIONS_FILE = "/content/drive/MyDrive/Colab Notebooks/flickr8k/captions.txt"
IMAGE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/flickr8k/Images/"

df = pd.read_csv(CAPTIONS_FILE, names=["image", "caption"], usecols=[0, 1])
df = df.groupby("image").first().reset_index()  # Keep only 1 caption per image
df = df.sample(n=2000, random_state=42)  # Select 2000 images randomly

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 2️⃣ Preprocess the Dataset

In [ ]:
def preprocess_data(example):
    if pd.isna(example["image"]) or pd.isna(example["caption"]):
        return {}

    image_path = os.path.join(IMAGE_FOLDER, example["image"])
    if not os.path.exists(image_path):
        print(f"Warning: Image not found - {image_path}")
        return {}

    try:
        image = Image.open(image_path).convert("RGB")
        encoding = processor(images=image, text=example["caption"], return_tensors="pt", padding="max_length", truncation=True)

        return {
            "pixel_values": encoding["pixel_values"].squeeze(),
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": encoding["input_ids"].squeeze(),
        }
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return {}

df.dropna(subset=["image", "caption"], inplace=True)
dataset = Dataset.from_pandas(df)
dataset = dataset.map(preprocess_data, remove_columns=["image", "caption"], batched=False)

train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

### 3️⃣ Define Optimized Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./blip_flickr8k",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=1000,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=1,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    logging_steps=200,
    learning_rate=3e-5,
    weight_decay=0.01,
    report_to="none",
    fp16=True,
    gradient_checkpointing=True,
    dataloader_num_workers=2,
    warmup_steps=100,
    save_on_each_node=False,
)


### 4️⃣ Load the BLIP Model & Data Collator

In [ ]:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

data_collator = DataCollatorForSeq2Seq(processor.tokenizer, model=model, padding=True)


### 5️⃣ Train the Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss,Validation Loss
200,4.823700,0.091851
400,0.054800,0.055504
600,0.034400,0.055461


Step,Training Loss,Validation Loss
200,4.823700,0.091851
400,0.054800,0.055504
600,0.034400,0.055461
800,0.024000,0.058299
1000,0.017000,0.059844


TrainOutput(global_step=1000, training_loss=0.9907985074520111, metrics={'train_runtime': 3136.179, 'train_samples_per_second': 2.551, 'train_steps_per_second': 0.319, 'total_flos': 4.745015523191292e+18, 'train_loss': 0.9907985074520111, 'epoch': 4.444444444444445})

### 6️⃣ Save Fine-tuned Model

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/blip_flickr8k_finetuned")
processor.save_pretrained("/content/drive/MyDrive/Colab Notebooks/blip_flickr8k_finetuned")


[]

### 7️⃣ Generate Captions to Compare Before & After Fine-tuning

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

model_finetuned = BlipForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/blip_flickr8k_finetuned").to(device)
processor_finetuned = BlipProcessor.from_pretrained("/content/drive/MyDrive/Colab Notebooks/blip_flickr8k_finetuned")

IMAGE_FILES = [
    "1000268201_693b08cb0e.jpg",
    "1001773457_577c3a7d70.jpg",
    "1002674143_1b742ab4b8.jpg",
    "1003163366_44323f5815.jpg",
    "1007129816_e794419615.jpg"
]

def generate_caption(model, processor, image_path):
    image = Image.open(image_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(**inputs)

    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

for image_file in IMAGE_FILES:
    image_path = os.path.join(IMAGE_FOLDER, image_file)

    ground_truth = df[df["image"] == image_file]["caption"].values
    ground_truth_caption = ground_truth[0] if len(ground_truth) > 0 else "N/A (Caption Missing)"

    caption_before = generate_caption(model, processor, image_path)
    caption_after = generate_caption(model_finetuned, processor_finetuned, image_path)

    image = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Ground Truth: {ground_truth_caption}", fontsize=10, wrap=True)
    plt.show()

    print(f"🖼️ Image: {image_file}")
    print(f"📌 Ground Truth: {ground_truth_caption}")
    print(f"📝 Caption Before Fine-Tuning: {caption_before}")
    print(f"✅ Caption After Fine-Tuning: {caption_after}")
    print("-" * 80)


Output hidden; open in https://colab.research.google.com to view.